In [1]:
import numpy as np
import h5py
'test'


'test'

In [2]:
def read_amira_binary_mesh(filename):
    with open(filename, 'rb') as f:
        raw_data = f.read()

    # Locate the first occurrence of "@1"
    first_marker_idx = raw_data.find(b'@1')
    if first_marker_idx == -1:
        raise ValueError("Could not find binary data section in Amira file.")

    # Locate the second occurrence of "@1"
    second_marker_idx = raw_data.find(b'@1', first_marker_idx + 2)
    if second_marker_idx == -1:
        raise ValueError("Could not find second binary data section in Amira file.")
    
    # The second @1 starts right after its occurrence
    data_start = second_marker_idx + 4  # Skip '@1\n'
    
    # Extract the binary data
    #print(raw_data[:data_start])
    binary_data = raw_data[data_start:]
    #print(binary_data[:100])

    # Define shape (from header: 512 x 512 x 1001, 2 channels per voxel)
    #lattice_shape = (512, 512, 1001, 2)
    lattice_shape = (1001, 512, 512, 2)

    # Read as float32 (Little-Endian)
    #float_data = np.frombuffer(binary_data, dtype=np.float32, count=1000)
    float_data = np.frombuffer(binary_data, dtype=np.float32)#, count=512*512)
    #print(float_data[:100])
    # Reshape
    float_data = float_data.reshape(lattice_shape)

    return float_data

In [7]:
files = ['0000.am', '4400.am']
directory = '../../data/'
for file in files:
    amira_data = read_amira_binary_mesh(directory + file)
    #print(amira_data.shape)
    print(amira_data[0,:,:,:].mean())

    h5_filename = directory + file.replace('.am', '.h5')  
    with h5py.File(h5_filename, 'w') as f:
        f.create_dataset(file, data=amira_data)

-3.9529707e-05
0.0049126064
